In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.8 MB/s eta 0:00:00


In [2]:
import optuna
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Load CIFAR-10 dataset

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
input_shape = x_train.shape[1:]

170498071/170498071 [==============================] - 2s 0us/step


# Define the search space for NAS using Optuna

In [7]:
def create_model(trial):
    model = Sequential()

    # Define the hyperparameters to optimize
    num_conv_layers = trial.suggest_int('num_conv_layers', 1, 5)
    num_filters = trial.suggest_categorical('num_filters', [Numf for Numf in range(1, 10)])
    num_dense_units = trial.suggest_categorical('num_dense_units', [Numd for Numd in range(256, 1025)])

    # Add convolutional layers
    for i in range(num_conv_layers):
        model.add(Conv2D(num_filters, (3, 3), activation='relu', padding='same', input_shape=input_shape))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.Dropout(0.25))
        #num_filters *= 2

    model.add(Flatten())

    # Add dense layers
    model.add(Dense(num_dense_units, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))

    # Output layer
    model.add(Dense(10, activation='softmax'))

    return model

# Define the objective function for Optuna optimization

In [9]:
def objective(trial):
    model = create_model(trial)

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=0)

    # Evaluate the model on the validation set
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)

    return accuracy

# Create an Optuna study and optimize the objective function

In [10]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2023-09-07 08:36:47,280] A new study created in memory with name: no-name-275f2ca0-70aa-4502-b806-4d1d996b9cf6
[I 2023-09-07 08:37:35,219] Trial 0 finished with value: 0.3919999897480011 and parameters: {'num_conv_layers': 2, 'num_filters': 6, 'num_dense_units': 556}. Best is trial 0 with value: 0.3919999897480011.
[I 2023-09-07 08:37:58,474] Trial 1 finished with value: 0.4372999966144562 and parameters: {'num_conv_layers': 2, 'num_filters': 6, 'num_dense_units': 491}. Best is trial 1 with value: 0.4372999966144562.
[I 2023-09-07 08:38:20,534] Trial 2 finished with value: 0.25609999895095825 and parameters: {'num_conv_layers': 4, 'num_filters': 9, 'num_dense_units': 329}. Best is trial 1 with value: 0.4372999966144562.
[I 2023-09-07 08:38:34,792] Trial 3 finished with value: 0.18960000574588776 and parameters: {'num_conv_layers': 1, 'num_filters': 6, 'num_dense_units': 265}. Best is trial 1 with value: 0.4372999966144562.
[I 2023-09-07 08:38:52,019] Trial 4 finished with value: 0.4

# Print the best hyperparameters and accuracy

In [11]:
best_params = study.best_params
best_accuracy = study.best_value
print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)

Best Hyperparameters: {'num_conv_layers': 2, 'num_filters': 4, 'num_dense_units': 971}
Best Accuracy: 0.4596000015735626
